In [ ]:
# Setting up google drive 
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import models as models
import five_fold_training
import torch
from torch import nn
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, ConcatDataset
from IPython import display
import math
import torchvision
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sb
import pickle
from torch.distributions.multivariate_normal import MultivariateNormal
  

In [ ]:


full_data_tens_path = '/content/drive/MyDrive/Data/MM_tens_data'
full_data_tens = torch.load(full_data_tens_path)

In [ ]:
path = '/content/drive/MyDrive/Data/MM_tens_data'
data = torch.load(path)

In [ ]:
net25b02 = models.BernoulliVAE2(input_size=204, 
      hidden_size1=196, 
      hidden_size2=128, 
      latent_size=25)
path = '/content/drive/MyDrive/Data/VAE25_beta0.2_batch128_fold3'
net25b02.load_state_dict(torch.load(path))



<All keys matched successfully>

In [ ]:
z, mu_e, logvar_e, std = net25b02.encode(full_data_tens)

In [ ]:
mean = z.mean(axis=0)
print(mean)
stdev = z.std(axis=0)
print(stdev)






tensor([-0.0240,  0.0213,  0.0792, -0.0253,  0.0361,  0.0245, -0.0112,  0.0078,
        -0.0046,  0.0012,  0.0228,  0.0058,  0.0049, -0.0260, -0.0158, -0.0338,
        -0.0373, -0.0475,  0.0673,  0.0056,  0.0199,  0.0095,  0.0158, -0.0317,
        -0.0058], grad_fn=<MeanBackward1>)
tensor([0.9737, 1.0102, 1.0071, 1.0021, 1.0328, 0.9970, 0.9889, 1.0302, 0.9910,
        1.0032, 0.9923, 0.9735, 1.0468, 1.0338, 0.9991, 1.0043, 1.1081, 0.9922,
        1.0081, 0.9928, 0.9955, 1.0068, 1.0169, 1.0508, 0.9900],
       grad_fn=<StdBackward0>)


tensor([-1.8665,  0.9868, -0.2071, -0.5763,  1.3356,  0.3262,  0.4814, -2.0291,
         0.2848, -0.5430,  1.6347, -0.5288, -0.2083, -0.6563, -0.8710, -0.4869,
         1.6406, -1.6940,  1.6110,  0.7124, -0.4539,  0.3215,  0.5525, -0.4677,
        -0.7799], grad_fn=<NormalBackward3>)

In [ ]:
gen_data = np.zeros(shape=(40469, 204))
gen_latents = np.zeros(shape=(40469, 25))
net25b02.eval()
for i in range(40469):
  rand_inp = torch.normal(mean=mean, std=stdev)
  gen_latents[i] = rand_inp.detach().numpy() 
  #print('rand_inp = ', rand_inp)

  with torch.no_grad():
    synth_data = net25b02.decode(rand_inp).numpy()
    synth_data = synth_data.round()
    gen_data[i] = synth_data
  #print("\nSynthetic generated data: ")
  #print(sd.round()) 



[[ 1.09261715 -1.26220179 -1.61260986 ... -2.1862452   0.53135592
   0.49387935]
 [ 0.92892164 -0.27572775  0.64086485 ...  0.1615043   0.79686242
   0.65418398]
 [ 0.07701731  0.87327027 -0.7623744  ... -0.67357844 -0.38631046
   1.33188105]
 ...
 [ 0.54881907 -0.31107366  0.71368396 ... -0.2170483   2.21058893
  -2.10989785]
 [ 0.4797031   0.02307747  0.13443999 ...  2.48850703 -0.98645109
   1.84127855]
 [-1.35192955  2.24970341 -0.34134066 ... -0.60033435  0.29601717
  -0.46649024]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
unique, counts = np.unique(gen_data, return_counts=True)
print(unique)
counts


[0. 1.]


array([7772864,  482812])

In [ ]:

#gen_out = net1_50.decode(gen_latents)
#gen_data = gen_out.round()

In [ ]:
#gen_data

In [ ]:
MM_real_lat04 = z
path = '/content/drive/MyDrive/Data/MM_real_latents_lat25_beta0.2'
torch.save(MM_real_lat04, path)

In [ ]:
MM_gen_data04 = gen_data
path = '/content/drive/MyDrive/Data/MM_gen_data_lat25_beta0.2'
torch.save(MM_gen_data04, path)

In [ ]:
MM_gen_lat04 = gen_latents
path = '/content/drive/MyDrive/Data/MM_gen_latents_lat25_beta0.2'
torch.save(MM_gen_lat04, path)